In [77]:
import pronouncing
import difflib

In [ ]:
def get_phonetic(word):
	p = pronouncing.phones_for_word(word.lower())
	return p[0] if p else None

def phrase_phonetic(phrase):
	phonetics = []
	for word in phrase.split():
		phonetic = get_phonetic(word)
		if phonetic:
			phonetics.append(phonetic)
	return " ".join(phonetics)

def phonetic_similarity(p1, p2):
	return difflib.SequenceMatcher(None, p1, p2).ratio()

def get_similar_words(word, threshold=.6):
	original_phonetic = get_phonetic(word)
	if not original_phonetic:
		return [word] # return just original if nothign

	candidates = set()
	# find rhymes
	for candidate in pronouncing.rhymes(word.lower()):
		candidate_phonetic = get_phonetic(candidate)
		if candidate_phonetic:
			similarity = phonetic_similarity(None, original_phonetic, candidate_phonetic)
		if similarity >= threshold:
			candidates.add(candidate)

	candidates.add(word)
	return list(candidates)

def phoneme_to_words(phoneme_str, threshold=0.6, max_num=None):
	# phoneme string and convert it into the most similar words
	candidates = set()
	for word, candidate_phone in pronouncing.pronunciations:
		candidate_phone = get_phonetic(word)
		if candidate_phone:
			similarity = difflib.SequenceMatcher(None, phoneme_str, candidate_phone).ratio()
			if similarity >= threshold:
				candidates.add(word)
				if max_num is not None:
					if len(candidates) > max_num:
						break


	return list(candidates)


In [78]:
def beam_search_segment(phoneme_seq, beam_width=10, phonetic_weight=1.0, word_threshold=0.6):
	# Each beam is a tuple: (segmentation_so_far, current_index, combined_score)
	beams = [("", 0, 0)]
	complete_beams = []

	phoneme_seq_list = phoneme_seq.split(" ")
	# Continue until there are no partial beams to extend.
	while beams:
		new_beams = []
		for seg_str, index, score in beams:
			if index == len(phoneme_seq_list):
				complete_beams.append((seg_str, score))
				continue

			# Try every possible split from index+1 to the end.
			for end in range(index + 1, len(phoneme_seq_list) + 1):
				candidate_phoneme = " ".join(phoneme_seq_list[index:end])
				candidate_words = phoneme_to_words(candidate_phoneme, threshold=word_threshold)
				if candidate_words:
					for word in candidate_words:
						new_seg = (seg_str + " " + word).strip()
						candidate_signature = phrase_phonetic(new_seg)
						ph_score = phonetic_similarity(phoneme_seq, candidate_signature)
						combined_score =  + phonetic_weight * ph_score
						new_beams.append((new_seg, end, combined_score))
		# Prune to the top beams.
		new_beams.sort(key=lambda x: x[2], reverse=True)
		beams = new_beams[:beam_width]

	# Return the complete segmentations sorted by score.
	complete_beams.sort(key=lambda x: x[1], reverse=True)
	return [seg for seg, score in complete_beams]


In [79]:
phrase = "hugh jass"
phoneme_seq = phrase_phonetic(phrase)
out = beam_search_segment(phoneme_seq)


In [80]:
out

['hugest',
 'hugus ass',
 "hewitt's ass",
 'humus ass',
 'hugus ass',
 "hewitt's ass",
 'hugest s',
 'hugest ess',
 'umass',
 'hugus ass sh',
 'hugus ass shh',
 "hewitt's ass sh",
 "hewitt's ass shh",
 'hugest ass sh',
 'hugest ass shh']